In [1]:
import warnings
warnings.filterwarnings('ignore', category= Warning)
import pandas as pd
import math
from sklearn.model_selection import train_test_split,KFold
import numpy as np
import os

In [15]:
os.chdir("C:/Users/yasar/Dropbox/Edvancer/ML_Python/$$___Python - Project--3___$$")

In [16]:
cf_train = pd.read_csv("counterfeit_train.csv")
cf_test = pd.read_csv("counterfeit_test.csv")

In [6]:
# 6818+1705

In [7]:
# cf_train.info()

In [32]:
# cf_test.info()

In [8]:
cf_test['Counterfeit_Sales'] = np.nan

In [9]:
cf_test.head()

,Medicine_ID,Counterfeit_Weight,DistArea_ID,Active_Since,Medicine_MRP,Medicine_Type,SidEffect_Level,Availability_rating,Area_Type,Area_City_Type,Area_dist_level,Counterfeit_Sales
0,HLZ81,NaN,Area027,1983,85.5328,Antibiotics,mild,0.112747,CityLimits,Tier 3,Medium,NaN
1,ECE94,13.45,Area045,2000,257.1460,OralContraceptives,mild,0.144446,DownTown,Tier 2,Unknown,NaN
2,SAD14,7.10,Area045,2000,98.1172,Antipyretics,mild,0.144221,DownTown,Tier 2,Unknown,NaN
3,EQV63,18.30,Area010,1996,135.3730,Tranquilizers,mild,0.100388,MidTownResidential,Tier 3,Unknown,NaN
4,AIR10,NaN,Area019,1983,112.8016,OralContraceptives,mild,0.022585,MidTownResidential,Tier 1,Small,NaN


In [33]:
# cf_train.isnull().sum()

In [34]:
# cf_test.isnull().sum()

### Pre_processing the data

In [41]:
cf_train['data'] = 'train'
cf_test['data'] = 'test'

In [42]:
cf_all = pd.concat([cf_train,cf_test],axis=0)

In [78]:
cf_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 1704
Data columns (total 22 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Medicine_ID                       8523 non-null   object 
 1   Counterfeit_Weight                7060 non-null   float64
 2   DistArea_ID                       8523 non-null   object 
 3   Active_Since                      8523 non-null   int64  
 4   Medicine_MRP                      8523 non-null   float64
 5   SidEffect_Level                   8523 non-null   object 
 6   Availability_rating               8523 non-null   float64
 7   Area_Type                         8523 non-null   object 
 8   Area_City_Type                    8523 non-null   object 
 9   Area_dist_level                   8523 non-null   object 
 10  Counterfeit_Sales                 6818 non-null   float64
 11  data                              8523 non-null   object 
 12  Medici

In [44]:
cf_all['data'].value_counts()

train    6818
test     1705
Name: data, dtype: int64

In [45]:
cf_all.isnull().sum()

Medicine_ID               0
Counterfeit_Weight     1463
DistArea_ID               0
Active_Since              0
Medicine_MRP              0
Medicine_Type             0
SidEffect_Level           0
Availability_rating       0
Area_Type                 0
Area_City_Type            0
Area_dist_level           0
Counterfeit_Sales      1705
data                      0
dtype: int64

In [54]:
cat_cols = cf_all.select_dtypes('object').columns

In [55]:
cat_cols = cat_cols.drop('data')

In [56]:
cat_cols

Index(['Medicine_ID', 'DistArea_ID', 'Medicine_Type', 'SidEffect_Level',
       'Area_Type', 'Area_City_Type', 'Area_dist_level'],
      dtype='object')

In [63]:
cf_all['Medicine_Type'].value_counts()

Antibiotics           1232
Hreplacements         1200
Antiseptics            910
OralContraceptives     856
Antipyretics           682
Cardiac                649
Mstablizers            648
Tranquilizers          520
Analgesics             445
Antimalarial           425
Antacids               251
Statins                214
MuscleRelaxants        169
Antifungal             148
Stimulants             110
Antiviral               64
Name: Medicine_Type, dtype: int64

In [97]:
for cat in cf_all.select_dtypes('object').columns:
    print(cat, ':', cf_all[cat].nunique())

Medicine_ID : 1557
Counterfeit_Weight : 416
data : 2


In [75]:
k = cf_all['Medicine_Type'].value_counts()
for val in k.axes[0][:10]:
    varname = 'Medicine_Type_'+val.replace(',','_').replace(' ','_')
    cf_all[varname] = np.where(cf_all['Medicine_Type']==val,1,0)
    
del cf_all['Medicine_Type']

Index(['Antibiotics', 'Hreplacements', 'Antiseptics', 'OralContraceptives',
       'Antipyretics'],
      dtype='object')

In [10]:
import matplotlib.pyplot as plt

In [11]:
pwd

'C:\\Users\\yasar\\Dropbox\\Edvancer\\ML_Python\\$$___Python - Project--3___$$'

In [82]:
for cat in ['DistArea_ID', 'SidEffect_Level','Area_Type', 'Area_City_Type', 'Area_dist_level']:
    temp = pd.get_dummies(cf_all[cat],prefix=cat,drop_first=True)
    cf_all = pd.concat([temp,cf_all],1)
    cf_all.drop([cat],1,inplace=True)

In [111]:
cf_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8523 entries, 0 to 1704
Data columns (total 35 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Area_dist_level_Medium            8523 non-null   uint8  
 1   Area_dist_level_Small             8523 non-null   uint8  
 2   Area_dist_level_Unknown           8523 non-null   uint8  
 3   Area_City_Type_Tier 2             8523 non-null   uint8  
 4   Area_City_Type_Tier 3             8523 non-null   uint8  
 5   Area_Type_DownTown                8523 non-null   uint8  
 6   Area_Type_Industrial              8523 non-null   uint8  
 7   Area_Type_MidTownResidential      8523 non-null   uint8  
 8   SidEffect_Level_mild              8523 non-null   uint8  
 9   DistArea_ID_Area013               8523 non-null   uint8  
 10  DistArea_ID_Area017               8523 non-null   uint8  
 11  DistArea_ID_Area018               8523 non-null   uint8  
 12  DistAr

In [109]:
cf_all['Counterfeit_Weight'].isnull().sum()

0

In [110]:
cf_all.isnull().sum()

Area_dist_level_Medium                 0
Area_dist_level_Small                  0
Area_dist_level_Unknown                0
Area_City_Type_Tier 2                  0
Area_City_Type_Tier 3                  0
Area_Type_DownTown                     0
Area_Type_Industrial                   0
Area_Type_MidTownResidential           0
SidEffect_Level_mild                   0
DistArea_ID_Area013                    0
DistArea_ID_Area017                    0
DistArea_ID_Area018                    0
DistArea_ID_Area019                    0
DistArea_ID_Area027                    0
DistArea_ID_Area035                    0
DistArea_ID_Area045                    0
DistArea_ID_Area046                    0
DistArea_ID_Area049                    0
Medicine_ID                            0
Counterfeit_Weight                     0
Active_Since                           0
Medicine_MRP                           0
Availability_rating                    0
Counterfeit_Sales                   1705
data            

In [108]:
cf_all['Counterfeit_Weight'].fillna(cf_all['Counterfeit_Weight'].mean(),inplace=True)
# cf_all['Counterfeit_Weight']= pd.to_numeric(cf_all['Counterfeit_Weight'],errors='coerce')

In [112]:
cf_all.shape

(8523, 35)

In [113]:
train_clean = cf_all[cf_all['data']=='train']
del train_clean['data']
test_clean = cf_all[cf_all['data']=='test']
test_clean.drop(['data','Counterfeit_Sales'],1,inplace=True)

In [114]:
train_clean.shape,test_clean.shape

((6818, 34), (1705, 33))

In [140]:
X = train_clean.drop(['Counterfeit_Sales','Medicine_ID'],1)
y = train_clean['Counterfeit_Sales']
X.shape,y.shape

((6818, 32), (6818,))

In [141]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=123)

In [142]:
from sklearn.linear_model import LinearRegression

In [176]:
lr = LinearRegression(fit_intercept=True)

In [180]:
lr.get_params()

{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': False}

In [177]:
lr.fit(X_train,y_train)

LinearRegression()

In [178]:
pred = lr.predict(X_val)

In [147]:
pred.shape,y_val.shape

((1364,), (1364,))

In [148]:
err = y_val-pred

In [152]:
from sklearn.metrics import mean_absolute_error
MAE = mean_absolute_error(y_val,pred)

In [153]:
MAE

845.6307788581305

In [154]:
Score = 1-(MAE/1660)

In [155]:
Score  #FOR LINEAR REGRESSION

0.4905838681577527

### DECISION TREE REGRESSOR

In [157]:
from sklearn.model_selection import RandomizedSearchCV

In [158]:
from sklearn.tree import DecisionTreeRegressor

In [165]:
params = {
    'criterion' : ['mse',None],
    'max_depth' : [None,5,10,15,20,30,40,50,70],
    'min_samples_leaf':[1,2,5,10,15,20],
    'min_samples_split':[2,5,10,15,20,25]
}

In [169]:
2*9*6*6*10

6480

In [166]:
clf = DecisionTreeRegressor()

In [170]:
random_search = RandomizedSearchCV(clf,param_distributions=params,cv=10,n_jobs=-1,n_iter=100,verbose=20)

In [183]:
random_search.fit(X_train,y_train)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0420s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0860s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.1s
[Parallel(n_jo

RandomizedSearchCV(cv=10, estimator=DecisionTreeRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['mse', None],
                                        'max_depth': [None, 5, 10, 15, 20, 30,
                                                      40, 50, 70],
                                        'min_samples_leaf': [1, 2, 5, 10, 15,
                                                             20],
                                        'min_samples_split': [2, 5, 10, 15, 20,
                                                              25]},
                   verbose=20)

In [164]:
clf.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'mse',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

In [172]:
def report(results, n_top=3):
    for i in range(1,n_top+1):
        candidates = np.flatnonzero(results['rank_test_score']== i)
        
        for candidate in candidates:
            print('Model with rank: {0}'.format(i))
            print('Mean Validation Score: {0:.8f} (std:{1:.3f})'.format(
                    results['mean_test_score'][candidate],
                    results['std_test_score'][candidate]))
            print("Parameters:{0}".format(results['params'][candidate]))
            print("")

In [184]:
report(random_search.cv_results_,5)

Model with rank: 1
Mean Validation Score: 0.59452833 (std:0.022)
Parameters:{'min_samples_split': 20, 'min_samples_leaf': 20, 'max_depth': 5, 'criterion': 'mse'}

Model with rank: 2
Mean Validation Score: 0.59396771 (std:0.023)
Parameters:{'min_samples_split': 5, 'min_samples_leaf': 15, 'max_depth': 5, 'criterion': 'mse'}

Model with rank: 3
Mean Validation Score: 0.59202097 (std:0.023)
Parameters:{'min_samples_split': 15, 'min_samples_leaf': 10, 'max_depth': 5, 'criterion': 'mse'}

Model with rank: 3
Mean Validation Score: 0.59202097 (std:0.023)
Parameters:{'min_samples_split': 2, 'min_samples_leaf': 10, 'max_depth': 5, 'criterion': 'mse'}

Model with rank: 5
Mean Validation Score: 0.59140420 (std:0.023)
Parameters:{'min_samples_split': 15, 'min_samples_leaf': 5, 'max_depth': 5, 'criterion': 'mse'}



In [185]:
random_search.best_estimator_

DecisionTreeRegressor(max_depth=5, min_samples_leaf=20, min_samples_split=20)

In [186]:
dtree_reg = random_search.best_estimator_

In [187]:
dtree_reg.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=5, min_samples_leaf=20, min_samples_split=20)

In [188]:
pred = dtree_reg.predict(X_val)

In [189]:
pred.shape, y_val.shape

((1364,), (1364,))

In [190]:
MAE = mean_absolute_error(y_val,pred)
MAE

769.9517934603456

In [191]:
Score = 1-(MAE/1660)
Score #For DecisionTress with RandomSearchCV

0.5361736183973822

In [193]:
test_clean.shape,X_train.shape

((1705, 33), (5454, 32))

In [194]:
X_test = test_clean.drop(['Medicine_ID'],1)

In [196]:
X_test.shape,X_val.shape

((1705, 32), (1364, 32))

In [197]:
actual_pred = dtree_reg.predict(X_test)

In [198]:
actual_pred.shape

(1705,)

In [199]:
submission=pd.DataFrame(list(zip(test_clean['Medicine_ID'],list(actual_pred))),
                       columns=['Medicine_ID','Counterfeit_Sales'])

In [200]:
submission.head()

,Medicine_ID,Counterfeit_Sales
0,HLZ81,1960.734353
1,ECE94,4030.213146
2,SAD14,1575.408237
3,EQV63,273.109048
4,AIR10,358.111404


In [201]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1705 entries, 0 to 1704
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Medicine_ID        1705 non-null   object 
 1   Counterfeit_Sales  1705 non-null   float64
dtypes: float64(1), object(1)
memory usage: 26.8+ KB


In [202]:
submission.to_csv('sample_submission.csv',index=False)